In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
path = 'unsplash-lite-dataset-25k-nature/'
documents = ['photos', 'keywords', 'collections', 'conversions']
datasets = {}

for doc in documents:
  files = glob.glob(path + doc + ".tsv*")

  subsets = []
  for filename in files:
    df = pd.read_csv(filename, sep='\t', header=0)
    subsets.append(df)

  datasets[doc] = pd.concat(subsets, axis=0, ignore_index=True)

In [4]:
datasets['photos'].count()

photo_id                          25000
photo_url                         25000
photo_image_url                   25000
photo_submitted_at                25000
photo_featured                    25000
photographer_username             25000
photographer_first_name           25000
photographer_last_name            23344
exif_camera_make                  22170
exif_camera_model                 22126
exif_iso                          21785
exif_aperture_value               21384
exif_focal_length                 21483
exif_exposure_time                21762
photo_location_name                9478
photo_location_latitude            6895
photo_location_longitude           6892
photo_location_country             8563
photo_location_city                5984
stats_views                       25000
stats_downloads                   25000
ai_description                    23607
ai_primary_landmark_name              0
ai_primary_landmark_latitude          0
ai_primary_landmark_longitude         0


In [10]:
datasets['keywords'].count()

photo_id                   2689739
keyword                    2689739
ai_service_1_confidence    2390469
ai_service_2_confidence     219507
suggested_by_user          2689739
dtype: int64

In [17]:
print(datasets['collections'].count())
print(len(datasets['collections']['collection_id'].unique()))

photo_id              1646597
collection_id         1646597
collection_title      1646575
photo_collected_at    1646597
dtype: int64
405906


In [12]:
datasets['conversions'].count()

converted_at          4075504
conversion_type       4075504
keyword               4075504
photo_id              4075504
anonymous_user_id     4075504
conversion_country    4068248
dtype: int64